In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('chorales/train/chorale_000.csv')

In [3]:
df

,note0,note1,note2,note3
0,74,70,65,58
1,74,70,65,58
2,74,70,65,58
3,74,70,65,58
4,75,70,58,55
...,...,...,...,...
187,70,65,62,46
188,70,65,62,46
189,70,65,62,46
190,70,65,62,46


In [4]:
import os

train_files = sorted([os.path.join('chorales', 'train', f) for f in os.listdir(os.path.join('chorales', 'train')) if f.endswith('.csv')])
valid_files = sorted([os.path.join('chorales', 'valid', f) for f in os.listdir(os.path.join('chorales', 'valid')) if f.endswith('.csv')])
test_files = sorted([os.path.join('chorales', 'test', f) for f in os.listdir(os.path.join('chorales', 'test')) if f.endswith('.csv')])

In [5]:
train_data = [pd.read_csv(f).values.tolist() for f in train_files]
valid_data = [pd.read_csv(f).values.tolist() for f in valid_files]
test_data = [pd.read_csv(f).values.tolist() for f in test_files]

Lowest note: 36 -> C1<br>
Highest note: 81 -> A5<br>
Silence: 0

In [6]:
train_data[0]

[[74, 70, 65, 58],
 [74, 70, 65, 58],
 [74, 70, 65, 58],
 [74, 70, 65, 58],
 [75, 70, 58, 55],
 [75, 70, 58, 55],
 [75, 70, 60, 55],
 [75, 70, 60, 55],
 [77, 69, 62, 50],
 [77, 69, 62, 50],
 [77, 69, 62, 50],
 [77, 69, 62, 50],
 [77, 70, 62, 55],
 [77, 70, 62, 55],
 [77, 69, 62, 55],
 [77, 69, 62, 55],
 [75, 67, 63, 48],
 [75, 67, 63, 48],
 [75, 69, 63, 48],
 [75, 69, 63, 48],
 [74, 70, 65, 46],
 [74, 70, 65, 46],
 [74, 70, 65, 46],
 [74, 70, 65, 46],
 [72, 69, 65, 53],
 [72, 69, 65, 53],
 [72, 69, 65, 53],
 [72, 69, 65, 53],
 [72, 69, 65, 53],
 [72, 69, 65, 53],
 [72, 69, 65, 53],
 [72, 69, 65, 53],
 [74, 70, 65, 46],
 [74, 70, 65, 46],
 [74, 70, 65, 46],
 [74, 70, 65, 46],
 [75, 69, 63, 48],
 [75, 69, 63, 48],
 [75, 67, 63, 48],
 [75, 67, 63, 48],
 [77, 65, 62, 50],
 [77, 65, 62, 50],
 [77, 65, 60, 50],
 [77, 65, 60, 50],
 [74, 67, 58, 55],
 [74, 67, 58, 55],
 [74, 67, 58, 53],
 [74, 67, 58, 53],
 [72, 67, 58, 51],
 [72, 67, 58, 51],
 [72, 67, 58, 51],
 [72, 67, 58, 51],
 [72, 65, 57

In [7]:
from music21 import stream, chord

chorale = train_data[20]
s = stream.Stream()
for row in chorale:
    s.append(chord.Chord([n for n in row if n], quarterLength=1))
s.show('midi')

### Preprocessing Data

In [8]:
import numpy as np

min_note, max_note = 36, 81
window_size, window_offset, batch_size = 32, 16, 32


def make_xy(chorales):
    # get segments of 33 chords with 16 chords offset between them
    windows = [c[i:i + window_size + 1] for c in chorales for i in range(0, len(c) - window_size, window_offset)]

    data = np.array(windows, dtype=int)
    # if note is 0, keep it, otherwise rescale notes from 36-81 to 1-46
    data = np.where(data==0, 0, data - min_note + 1)
    # make the range 0-46 in total
    data = np.clip(data, 0, max_note - min_note + 1)

    flat = data.reshape(data.shape[0], -1)

    # return every chord except the last one for X (32 notes) and everything except the first one for Y (32 notes)
    return flat[:, :-1], flat[:, 1:]


X_train, Y_train = make_xy(train_data)
X_valid, Y_valid = make_xy(valid_data)
X_test,  Y_test  = make_xy(test_data)

In [9]:
X_train.shape  # 3111 times 32 chords with 4 notes each

(3111, 131)

In [10]:
Y_train.shape  # same but shifted by one

(3111, 131)

### Training Model

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Conv1D, BatchNormalization, LSTM, Dropout
from tensorflow.keras.optimizers import Nadam

2025-08-13 14:17:32.417476: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-13 14:17:32.427751: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-13 14:17:32.437842: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755087452.453131 1112079 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755087452.457436 1112079 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755087452.472060 1112079 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [12]:
train_notes = set([z for x in train_data for y in x for z in y])
test_notes = set([z for x in test_data for y in x for z in y])
valid_notes = set([z for x in valid_data for y in x for z in y])

num_notes = len(set.union(train_notes, test_notes, valid_notes))
num_notes

47

In [13]:
model = Sequential()

# Embedding layer so the model can learn representations of the notes (learns vector representation of notes)
# Integers have no geometry, by learning embeddings, we can keep "nearby" notes close in vector space (e.g., same pitch-class, close octaves etc.)
# Since we don't have too many notes, we can use a small dimensionality like 5
model.add(Embedding(input_dim=num_notes, output_dim=5, input_shape=[None]))
# 1D convs allow us to extract temporal patterns in parallel (unlike RNN layers)
# 1D convs slide 1D kernels / filters over our feature vector to learn temporal patterns
# Padding causal means we cannot look ahead, so we keep causality
model.add(Conv1D(32, kernel_size=2, padding="causal", activation="relu"))  # here 32 filters of size 2
# Batch norms after each conv keep activations well-scaled and consistent across the whole stack of layers
# Counteracts vanishing / exploding gradients, allows for higher stable learning rates and faster training
model.add(BatchNormalization())
# Dilation rate means how much we look back
# For example kernel size 2 and dilation rate of one means look at t and t-1
# Kernel size 2 and dilation rate of two means look at t and t-2
# Kernel size 2 and dilation rate of 16 means look at t and t-16
# Stacking these increasing dilation rates allows us to efficiently cover short, medium and longer history
# We grow the receptive field without using a lot of model parameters
# We indirectly connect notes that are up to 32 positions apart
model.add(Conv1D(48, kernel_size=2, padding="causal", activation="relu", dilation_rate=2))
model.add(BatchNormalization())
# Also, since we increase dilation rate, each conv layer sees a wider time span
# More kernels / filters allow us to capture more kinds of patterns
# If we don't increase this, we could end up with a bottleneck here
model.add(Conv1D(64, kernel_size=2, padding="causal", activation="relu", dilation_rate=4))
model.add(BatchNormalization())
model.add(Conv1D(96, kernel_size=2, padding="causal", activation="relu", dilation_rate=8))
model.add(BatchNormalization())
model.add(Conv1D(128, kernel_size=2, padding="causal", activation="relu", dilation_rate=16))
model.add(BatchNormalization())
# Just a bit of regularization here so the model does not rely too much on individual features
model.add(Dropout(0.05))
# The conv layers summarized local and mid-range context into richer features for us
# The LSTM now only has to track longer structure of the music
# Doing LSTM first would lead to discovering local AND long patterns, which is slower and harder to optimize
# Convs first is cheaper and parallelizable
# We can basically combine the layers to look back 32 tokens (like a binary system numbers)
model.add(LSTM(256, return_sequences=True))
# Finally a dense layer to project the LSTM output to logits for each possible note
model.add(Dense(num_notes, activation='softmax'))

model.summary()

/home/neuralnine/Documents/Programming/NeuralNine/Prepared/AI Bach Music/.venv/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:100: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
E0000 00:00:1755087454.143092 1112079 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1755087454.151508 1112079 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, None, 5)        │           235 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, None, 32)       │           352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, None, 32)       │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, None, 48)       │         3,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, None, 48)       │           192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, None, 64)       │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, None, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, None, 96)       │        12,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, None, 96)       │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, None, 128)      │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, None, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, None, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, None, 256)      │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, None, 47)       │        12,079 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 454,794 (1.73 MB)

 Trainable params: 454,058 (1.73 MB)

 Non-trainable params: 736 (2.88 KB)

In [14]:
optimizer = Nadam(learning_rate=1e-3)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=20, validation_data=[X_valid, Y_valid], batch_size=batch_size)

Epoch 1/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 15s 124ms/step - accuracy: 0.3329 - loss: 2.6009 - val_accuracy: 0.0580 - val_loss: 3.8693
Epoch 2/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 12s 120ms/step - accuracy: 0.7600 - loss: 0.9114 - val_accuracy: 0.1246 - val_loss: 3.7618
Epoch 3/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 12s 120ms/step - accuracy: 0.7938 - loss: 0.7406 - val_accuracy: 0.1810 - val_loss: 2.8922
Epoch 4/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 12s 120ms/step - accuracy: 0.8101 - loss: 0.6615 - val_accuracy: 0.3713 - val_loss: 2.1998
Epoch 5/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 12s 121ms/step - accuracy: 0.8232 - loss: 0.6014 - val_accuracy: 0.5090 - val_loss: 1.5995
Epoch 6/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 12s 122ms/step - accuracy: 0.8318 - loss: 0.5647 - val_accuracy: 0.7126 - val_loss: 0.9518
Epoch 7/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 12s 122ms/step - accuracy: 0.8411 - loss: 0.5242 - val_accuracy: 0.7977 - val_loss: 0.6811
Epoch 8/20
98/98 ━━━━━━━━━━━━━━━━━━━━ 12s 123ms/step - accuracy: 0.8492 - loss: 0.4968 - val_accu

#### Generate Music with Model

In [15]:
import numpy as np


def sample_next_note(probs):
    probabilities = np.asarray(probs, dtype=float)  # probabilities for each note to be the next
    
    probs_sum = probabilities.sum()  # get the sum for normalization

    # if the probability sum is zero, negative or infinite -> just return the note with the highest probability
    if probs_sum <= 0 or not np.isfinite(probs_sum):
        return int(np.argmax(probabilities))

    probabilities /= probs_sum # otherwise normalize the probabilities to be between 0 and 1
    return np.random.choice(len(probabilities), p=probabilities)  # randomly select a note based on probability
    
    
def generate_chorale(model, seed_chords, length):
    token_sequence = np.array(seed_chords, dtype=int)  # get starting chords / notes
    token_sequence = np.where(token_sequence == 0, token_sequence, token_sequence - 36 + 1)  # map all notes to 0-46 as in training
    token_sequence = token_sequence.reshape(1, -1)

    # we generate note by note, not chord by chord
    for _ in range(length * 4):
        next_token_probabilities = model.predict(token_sequence, verbose=0)[0, -1]  # get probabilities for next note from model
        next_token = sample_next_note(next_token_probabilities)  # sample from probabilities with sample function
        token_sequence = np.concatenate([token_sequence, [[next_token]]], axis=1) 
        
    token_sequence = np.where(token_sequence == 0, token_sequence, token_sequence + 36 - 1)   # map to MIDI (0 & 36-81)
    
    return token_sequence.reshape(-1, 4)

In [16]:
# Initial chords (seed) 
seed_chords = test_data[2][:8]

chorale = seed_chords
s = stream.Stream()
for row in chorale:
    s.append(chord.Chord([n for n in row if n], quarterLength=1))
s.show('midi')

In [17]:
# Complete actual chorale (ground truth)
seed_chords = test_data[2]

chorale = seed_chords
s = stream.Stream()
for row in chorale:
    s.append(chord.Chord([n for n in row if n], quarterLength=1))
s.show('midi')

In [18]:
# Generate new chords based on initial two chords (8 notes)
# Results can be better and more creative by introducing temperature, top_p etc.
seed_chords = test_data[2][:8]
new_chorale = generate_chorale(model, seed_chords, 56)

In [19]:
new_chorale

array([[73, 68, 61, 53],
       [73, 68, 61, 53],
       [73, 68, 61, 53],
       [73, 68, 61, 53],
       [69, 66, 61, 54],
       [69, 66, 61, 54],
       [69, 66, 61, 54],
       [69, 66, 61, 54],
       [71, 61, 59, 56],
       [71, 61, 59, 56],
       [71, 63, 59, 54],
       [71, 63, 59, 54],
       [71, 64, 59, 56],
       [71, 64, 59, 56],
       [71, 64, 59, 58],
       [71, 64, 59, 58],
       [67, 64, 61, 58],
       [67, 64, 61, 58],
       [64, 64, 61, 58],
       [64, 64, 61, 58],
       [64, 64, 61, 58],
       [64, 64, 61, 58],
       [66, 64, 61, 58],
       [66, 64, 61, 58],
       [66, 63, 59, 58],
       [66, 63, 59, 58],
       [66, 63, 59, 58],
       [66, 63, 59, 58],
       [66, 63, 59, 58],
       [66, 63, 59, 57],
       [66, 63, 59, 57],
       [66, 63, 59, 59],
       [66, 63, 59, 54],
       [66, 63, 59, 54],
       [66, 63, 59, 54],
       [66, 63, 59, 54],
       [66, 63, 59, 54],
       [66, 63, 59, 55],
       [67, 64, 59, 55],
       [67, 64, 59, 55],


In [20]:
# listen to generated piece

chorale = new_chorale.tolist()
s = stream.Stream()
for row in chorale:
    s.append(chord.Chord([n for n in row if n], quarterLength=1))
s.show('midi')

In [21]:
def generate_random_chorale(length, rest_probability=0.2, pitch_low=36, pitch_high=81, seed=None):
    rng = np.random.default_rng(seed)  # random number generator
    random_pitches = rng.integers(pitch_low, pitch_high + 1, size=(length, 4))  # generate random notes

    # some masking to have both silence and random pitches
    rest_mask = rng.random((length, 4)) < float(rest_probability)
    chorale = np.where(rest_mask, 0, random_pitches).astype(int)
    
    return chorale

In [22]:
# listen to completely random music to compare the quality to what our model generated
chorale = generate_random_chorale(56).tolist()
s = stream.Stream()
for row in chorale:
    s.append(chord.Chord([n for n in row if n], quarterLength=1))
s.show('midi')

In [23]:
# save model
model.save('bach_generation_conv1d_lstm.keras')

In [24]:
from tensorflow import keras

loaded_model = keras.models.load_model("bach_generation_conv1d_lstm.keras")

In [25]:
new_chorale = generate_chorale(loaded_model, seed_chords, 56)

In [26]:
new_chorale

array([[73, 68, 61, 53],
       [73, 68, 61, 53],
       [73, 68, 61, 53],
       [73, 68, 61, 53],
       [69, 66, 61, 54],
       [69, 66, 61, 54],
       [69, 66, 61, 54],
       [69, 66, 61, 54],
       [68, 64, 61, 61],
       [68, 64, 61, 61],
       [68, 64, 61, 57],
       [68, 64, 61, 57],
       [69, 66, 62, 59],
       [69, 66, 62, 59],
       [74, 68, 62, 59],
       [74, 68, 62, 59],
       [74, 68, 62, 59],
       [74, 68, 62, 59],
       [74, 69, 62, 54],
       [74, 69, 62, 54],
       [74, 71, 59, 56],
       [74, 71, 59, 56],
       [73, 69, 61, 57],
       [73, 69, 61, 57],
       [73, 69, 61, 57],
       [73, 69, 61, 57],
       [74, 69, 62, 56],
       [74, 69, 62, 56],
       [76, 67, 61, 56],
       [76, 67, 61, 56],
       [76, 66, 61, 54],
       [76, 66, 61, 54],
       [75, 66, 62, 54],
       [75, 66, 62, 54],
       [76, 66, 62, 54],
       [76, 66, 62, 54],
       [76, 66, 62, 59],
       [76, 66, 59, 59],
       [75, 66, 59, 59],
       [75, 66, 59, 59],
